In [1]:
from pulp import value
# However we do not expect the reader to add that folder to the env variable,
# therefore we manually load it temporarily in each notebook.
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pandas as pd
from timeit import default_timer as timer
from collections import defaultdict
import datetime
from modules.config import (
    PATH_SCENARIOS_REDUCED,
    PATH_DISTANCES,
    PATH_SCENARIO_PROBABILITY,
    PATH_INITIAL_ALLOCATION,
    PATH_RESULTS_SUMMARY,
)
from modules.stochastic_program.factory import StochasticProgramFactory



### Prepare scenario data
Make scenarios complete cross product

In [3]:
scenarios = pd.read_pickle(PATH_SCENARIOS_REDUCED)

In [4]:
hex_ids = {*scenarios.index.get_level_values('start_hex_ids').unique()}
hex_ids = list(hex_ids.union({*scenarios.index.get_level_values('end_hex_ids').unique()}))

In [5]:
complete_index = pd.MultiIndex.from_product(
    [
        scenarios.index.get_level_values('scenarios').unique(),
        pd.Index(hex_ids, name="start_hex_ids"),
        pd.Index(hex_ids, name="end_hex_ids"),
        scenarios.index.get_level_values('time').unique(),
        scenarios.index.get_level_values('vehicle_types').unique(),
    ]
)

In [6]:
scenarios = scenarios.reindex(complete_index, fill_value=0)

Find tree structure for non-anticipativity constraints

In [7]:
scenarios_unstacked = scenarios.unstack(level=['start_hex_ids', 'end_hex_ids', 'vehicle_types']) \
             .swaplevel() \
             .sort_index()

In [8]:
node_df = pd.DataFrame(index=scenarios_unstacked.index)
node_df['node'] = 0
node_df['prev_node'] = 0

In [9]:
times = scenarios_unstacked.index.get_level_values('time').unique()
scenario_ids = scenarios_unstacked.index.get_level_values('scenarios').unique()

nodes = []
nodes_counter = 0
groups = [{} for _ in range(len(times))]
for i, time in enumerate(times):
    prev_groups = groups[i-1] if i != 0 else {-1: list(scenario_ids)}

    found_ids = []

    values = scenarios_unstacked.loc[(time)].values

    for scenario_id in scenario_ids:
        if scenario_id in found_ids:
            continue
        current_group = list((values == values[scenario_id]).all(axis=1).nonzero()[0])
        found_ids += current_group
        for prev_group_id, prev_group in prev_groups.items():
            group = [s_id for s_id in current_group if s_id in prev_group]

            if(not group):
                continue

            groups[i][nodes_counter] = group
            for s_id in group:
                node_df.loc[(time, s_id), 'node'] = nodes_counter
                node_df.loc[(time, s_id), 'prev_node'] = prev_group_id
            nodes_counter += 1

## Get Model Parameters


In [10]:
demand = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(dict))))

In [11]:
start = timer()

for _, row in scenarios.reset_index().iterrows():
    demand[row.start_hex_ids][row.end_hex_ids][row.time.hour][row.vehicle_types][row.scenarios] = row.demand

end = timer()
print(f"Successfully converted dataframe into dictionary in {(end - start):.2f} seconds")

In [ ]:
distances = pd.read_pickle(PATH_DISTANCES)

In [ ]:
cost = defaultdict(lambda: defaultdict(dict))
profit = defaultdict(lambda: defaultdict(dict))

In [ ]:
start = timer()

vehicle_types = list(scenarios.reset_index()['vehicle_types'].unique())

for _, row in distances.reset_index().iterrows():
    for vehicle_type in vehicle_types:
        cost[row.start_hex_id][row.end_hex_id][vehicle_type] = row[f"cost_{vehicle_type}"]
        profit[row.start_hex_id][row.end_hex_id][vehicle_type] = row[f"profit_{vehicle_type}"]

end = timer()

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
probabilities = pd.read_pickle(PATH_SCENARIO_PROBABILITY)

In [ ]:
factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df)

_convert_probabilities finished in 0.00 seconds
_convert_distances finished in 0.05 seconds
_convert_demand finished in 2.79 seconds
_convert_nodes finished in 0.00 seconds
_convert_parameters finished in 2.85 seconds
_set_max_demand finished in 0.17 seconds


In [ ]:
FLEET_CAPACITY = {
    "kick_scooter": 200,
    "bicycle": 100,
    "car": 50,
}
factory.set_initial_allocation(FLEET_CAPACITY)

set_initial_allocation finished in 0.01 seconds


In [ ]:
stochastic_program = factory.create_stochastic_program()

create_stochastic_program finished in 0.00 seconds


In [ ]:
stochastic_program.relocations_disabled = True

In [ ]:
stochastic_program.create_model()

create_variables finished in 0.31 seconds
create_objective_function finished in 0.65 seconds
create_demand_constraints finished in 0.55 seconds
create_relocation_binary_constraints finished in 0.02 seconds
create_big_u_sum_constraints finished in 0.09 seconds
create_unfulfilled_demand_binary_constraints finished in 0.03 seconds
create_no_refused_demand_constraints finished in 0.02 seconds
create_max_trips_constraints finished in 0.21 seconds
create_vehicle_movement_constraints finished in 0.22 seconds
create_initial_allocation_constraints finished in 0.00 seconds
create_non_anticipativity_constraints finished in 0.26 seconds
create_constraints finished in 1.40 seconds
create_model finished in 2.76 seconds


In [ ]:
stochastic_program.solve(logPath='./log')

/home/moritz/miniconda3/envs/VR/lib/python3.8/site-packages/pulp/apis/coin_api.py:146: UserWarning: `logPath` argument replaces `msg=1`. The output will be redirected to the log file.
  warnings.warn('`logPath` argument replaces `msg=1`. The output will be redirected to the log file.')


Status: Optimal
Optimal Value of Objective Function:  2474.13250701685
Runtime without preprocessing: 2.06 seconds
solve finished in 3.38 seconds


In [ ]:
stochastic_program.get_results_by_region_df().sum()

get_results_by_region_df finished in 0.02 seconds


accumulated_unfulfilled_demand    28556
has_unfulfilled_demand              478
has_remaining_vehicles             1955
n_vehicles                        16800
dtype: int64

In [ ]:
capacities = [{
    "kick_scooter": 200,
    "bicycle": 100,
    "car": 50,
},{
    "kick_scooter": 1000,
    "bicycle": 500,
    "car": 250,
},{
    "kick_scooter": 2000,
    "bicycle": 1000,
    "car": 500,
}]


In [ ]:
results = []

factory = StochasticProgramFactory(scenarios, distances, probabilities, node_df)
factory.include_methods = [None]
for capacity in capacities:
    factory.set_initial_allocation(capacity)

    stochastic_program = factory.create_stochastic_program()
    stochastic_program.include_methods = ['solve']

    for relocations_disabled in [False, True]:
        for non_anticipativity_disabled in [False, True]:
            stochastic_program.relocations_disabled = relocations_disabled
            stochastic_program.non_anticipativity_disabled = non_anticipativity_disabled
            stochastic_program.create_model()
            stochastic_program.solve()

            results.append({
                **stochastic_program.get_summary(),
                **capacity,
                'relocations_disabled': relocations_disabled,
                'non_anticipativity_disabled': non_anticipativity_disabled,
            })
            print('\n')

In [ ]:
results_df = pd.DataFrame.from_dict(results)

NameError: name 'results' is not defined

In [ ]:
results_df

NameError: name 'results_df' is not defined

In [ ]:
os.makedirs(os.path.dirname(PATH_RESULTS_SUMMARY), exist_ok=True)
results_df.to_pickle(PATH_RESULTS_SUMMARY)


NameError: name 'results_df' is not defined

In [ ]:
unfulfilled_demand = stochastic_program.get_results_by_tuple_df()['unfulfilled_demand'].to_frame()

get_results_by_tuple_df finished in 0.21 seconds


In [ ]:
unfulfilled_demand = unfulfilled_demand.reset_index()
unfulfilled_demand['time'] = unfulfilled_demand['time'].map(lambda hour: datetime.time(hour=hour))
unfulfilled_demand = unfulfilled_demand.set_index(['start_hex_ids', 'end_hex_ids', 'time', 'vehicle_types', 'scenarios'])

In [ ]:
unfulfilled_demand = unfulfilled_demand.reorder_levels(
    ["scenarios", "start_hex_ids", "end_hex_ids", "time", "vehicle_types"]
)